In [23]:
import sys
sys.path.append('/Users/Ziqi/Desktop/developer/mgwr/')
from mgwr.gwr import GWR
from mgwr.FastGWR import FastGWR
from mgwr.sel_bw import Sel_BW
import pandas as pd
import numpy as np

In [35]:
zillow = pd.read_csv("zillow_5k.csv",sep=',')
zillow.head()
y = zillow.value.as_matrix().reshape(-1,1)
X = zillow.iloc[:,3:].as_matrix()
k = zillow.shape[1]
u = zillow.utmX
v = zillow.utmY
n = zillow.shape[0]
coords = list(zip(u,v))

In [3]:
%%time
fast_rslt = FastGWR(coords,y,X).fit(nproc=4)

mpiexec -n 4 python /Users/Ziqi/Desktop/developer/gwr-hpc/FastGWR_mpi.py -data temp_data.csv -out fastGWRResults.csv
CPU times: user 148 ms, sys: 24.8 ms, total: 173 ms
Wall time: 30.3 s


In [33]:
%%time
fast_rslt = FastGWR(coords,y,X,bw=122).fit(nproc=2)

mpiexec -n 2 python /Users/Ziqi/Desktop/developer/gwr-hpc/FastGWR_mpi.py -data temp_data.csv -out fastGWRResults.csv -bw 122
CPU times: user 175 ms, sys: 30.2 ms, total: 205 ms
Wall time: 4.23 s


In [30]:
%%time 
#bw = Sel_BW(coords,y,X).search(bw_min=45)
rslt = GWR(coords,y,X,bw=122).fit()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/Ziqi/anaconda/lib/python3.5/site-packages/IPython/core/magics/execution.py", line 1238, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 2, in <module>
  File "/Users/Ziqi/Desktop/developer/mgwr/mgwr/gwr.py", line 255, in __init__
    self.W = self._build_W(fixed, kernel, coords, bw)
  File "/Users/Ziqi/Desktop/developer/mgwr/mgwr/gwr.py", line 265, in _build_W
    W = ak[kernel](coords, bw, points, self.dmat, self.sorted_dmat,spherical=self.spherical)
  File "/Users/Ziqi/Desktop/developer/mgwr/mgwr/kernels.py", line 29, in adapt_bisquare
    w = _Kernel(coords, fixed=False, k=nn-1, function='bisquare', points=points, dmat=dmat,sorted_dmat=sorted_dmat,spherical=spherical)
  File "/Users/Ziqi/Desktop/developer/mgwr/mgwr/kernels.py", line 102, in __init__
    self.dmat = cdist(self.data, self.data, self.spherical)
  File "/Users/Ziqi/Desktop/developer/mgwr/mgwr/kernels.py", line 68, in cdist
    dmat = cdist_scipy(coords

KeyboardInterrupt: 

In [14]:
#tr_S
print(rslt.tr_S)
print(fast_rslt.tr_S)

524.111964802687
524.111964802687


In [15]:
#AICc
print(rslt.aicc)
print(fast_rslt.aicc)

61866.751504274434
61866.751504274434


In [16]:
#Sigma2
print(rslt.sigma2)
print(fast_rslt.sigma2)

12227.724215085895
12227.724215085893


In [17]:
np.allclose(fast_rslt.params, rslt.params)

True

In [18]:
np.allclose(fast_rslt.bse, rslt.bse)

True

In [19]:
np.allclose(fast_rslt.predy, rslt.predy.reshape(-1))

True